In [365]:
from __future__ import print_function, division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from ggplot import *
import json

import os
from os.path import join
print('Current directory: {}'.format(os.getcwd()))
import sys
sys.path.append('..')
from glob import glob
from pprint import pprint
from time import time

DATA_PATH = '../../bucket/data/'
RESULTS_PATH = '../../bucket/results/'

Current directory: /Users/hmourit/Documents/0project/MScThesis/notebooks


# Latex stuff

In [366]:
import re
import subprocess

def mean_pm_std(mean, std):
    return '{:1.2f}$\pm${:1.2f}'.format(mean, std)
mean_pm_std = np.vectorize(mean_pm_std)

def pbcopy(data): 
    p = subprocess.Popen(['pbcopy'], stdin=subprocess.PIPE) 
    p.stdin.write(data) 
    p.stdin.close() 
    retcode = p.wait()    

def bold_center(cell):
    return r'\mc{1}{c}{\textbf{' + cell + '}}'

def process_latex(tabular, remove_lines=None):
    if remove_lines and not isinstance(remove_lines, list):
        remove_lines = [remove_lines]
    
    purged_tabular = []
    for line in tabular.split('\n'):
        if r'\begin{tabular}' in line:
            begin_tabular = line.split('}{')
            line = begin_tabular[0] + '}'
            line += '{' + begin_tabular[1].replace('l', 'r')
            purged_tabular.append(line)
        elif remove_lines and not any(re.match(pattern, line) for pattern in remove_lines):
            purged_tabular.append(line)
    tabular = '\n'.join(purged_tabular)
        
    table_env_begin = r"""\begin{table}[!h]
                       \centering
                       \footnotesize
                       """.replace(' ', '')
    table_env_end = r"""\caption{caption:placeholder}
                     \label{tab:placeholder}
                     \end{table}
                     """.replace(' ', '')
    
    return table_env_begin + tabular + table_env_end
    
CAPTION_PLACEHOLDER = r'caption:placeholder'
REMOVE_N_FEATURES = r'^n\\_features*'

TABLES_FOLDER = '~/Dropbox/MSc Thesis/tables'

def save_table(tex, filename, caption=None, folder=TABLES_FOLDER):
    folder = os.path.expanduser(folder)
    tex = tex.replace('tab:placeholder', 'tab:{}'.format(filename.split('.')[0]))
    if caption is not None:
        tex = tex.replace(CAPTION_PLACEHOLDER, caption)
    with open(os.path.join(folder, filename), 'w') as f:
        f.write(tex)
        
def my_replace(tex, name=None, src=None, dst=None, effects=None):
    name_map = {
        'en': (' en ', 'EN'),
        'svm': ('svm\_linear\_kernel', 'SVM'),
        'svm k': ('svm\_linear', 'SVM'),
        'l1 svm': ('svm\_linear\_l1', 'L1-SVM'),
        'clf': ('clf', 'CLF'),
        'pm': (r'\$\textbackslashpm\$', r'$\pm$'),
        'IG1': (r'infogain\_10', 'IG D1'),
        'IG2': (r'infogain\_exp', 'IG D2'),
        'anova': ('anova', 'ANOVA'),
        'mrmr': ('mrmr', 'mRMR'),
        'rfe': ('rfe', 'RFE'),
        'chi2': ('chi2', 'Chi2'),
        'stg': ('superior temporal gyrus', 'STG'),
        'wb': ('whole blood', 'WB'),
        'cer': ('cerebellum', 'CER'),
        'fc': ('frontal cortex', 'FC'),
        'ec': ('entorhinal cortex', 'EC')
    }
    
    effects_map = {
        '2': r'\mc{2}{c}{%s}',
        '2r': r'\multirow{2}{*}{%s}',
        '3r': r'\multirow{3}{*}{%s}',
        'b': r'\textbf{%s}',
        'c': r'\mc{1}{c}{%s}'
    }
    
    if name is not None:
        src = name_map[name][0]
        dst = name_map[name][1]
    
    if effects is not None:
        
        for effect in effects:
            dst = effects_map[effect] % dst
        
    return tex.replace(src, dst)     
        
def batch_replace(tex, replacements):
    for name, effects in replacements:
        tex = my_replace(tex, name=name, effects=effects)
    return tex

def add_clines(tex, clines):
    if not isinstance(clines, list):
        clines = iter([clines])
    else:
        clines = iter(clines)
    
    new_tex = []
    l, a, b = next(clines)
    i = 0
    for line in tex.split('\n'):
        new_tex.append(line)
        if line.endswith(r'\\'):
            i += 1
        rules = ''
        while l == i:
            rules += r'\cmidrule(rl){%d-%d}' % (a, b)
            try:
                l, a, b = next(clines)
            except StopIteration:
                l = -1
        if rules:
            new_tex.append(rules)
    
    return '\n'.join(new_tex)

# Useful constants

In [367]:
MDD = 'mdd_raw37'
EPI = 'epi_ad'
AD = 'ad.disease.status'

STG = 'superior temporal gyrus'
WB = 'whole blood'
CER = 'cerebellum'
FC = 'frontal cortex'
EC = 'entorhinal cortex'

LIN_SVM = 'svm_linear'
KER_SVM = 'svm_linear_kernel'
L1_SVM = 'svm_linear_l1'

GOOD_CLF_FILTER = ("(clf == 'en' & filter == ['rfe', 'chi2', 'infogain_exp']) "
                   "| (clf == ['svm_linear', 'svm_linear_kernel'] "
                      "& filter == ['anova', 'rfe', 'infogain_10', 'mrmr', 'chi2'])")

# Accuracy

In [448]:
from sklearn.metrics import accuracy_score

result_files = glob(join(RESULTS_PATH, '*_*.json'))
result_files = [x for x in result_files if os.path.basename(x).startswith(('anova', 'infogain', 'rfe', 'chi2', 'mrmr'))]

results = []
for f in result_files:
    exp_id = f.split('_')[-1].rstrip('.json')
    try:
        d = json.load(open(join(RESULTS_PATH, f), 'r'))
    except ValueError as e:
        if os.path.getsize(f) == 0:
            e = 'File size is 0. Removing.'
            os.remove(f)
        print('{} -> {}'.format(f, e))
    base = {'exp_id': exp_id}
    left = []
    for k, v in d.items():
        if k != 'experiments':
            base[k] = v
    if os.path.basename(f).startswith('mrmr'):
        base['filter'] = 'mrmr'
    elif os.path.basename(f).startswith('rfe'):
        base['filter'] = 'rfe'
    for exp in d['experiments']:
        it = exp['iteration']
        if 'subsets' not in exp:
            print('{} -> Field "subsets" not found.'.format(f))
            continue
        for s in exp['subsets']:
            train = accuracy_score(s['train']['y_true'], s['train']['y_pred'])
            test = accuracy_score(s['test']['y_true'], s['test']['y_pred'])
            if 'n_features' in s:
                n_features = s['n_features']
            else:
                n_features = len(s['features'])   
            results.append(dict(base, iteration=it, train=train, test=test, n_features=n_features))    

df = pd.DataFrame(results)

../../bucket/results/anova_-800824884543483809.json -> Field "subsets" not found.
../../bucket/results/anova_1773782541069684787.json -> Field "subsets" not found.
../../bucket/results/infogain_10_2347697712435705707.json -> Field "subsets" not found.
../../bucket/results/infogain_exp_-905003234740235046.json -> Field "subsets" not found.
../../bucket/results/infogain_exp_7158626770039509150.json -> Field "subsets" not found.


# Robustness

In [337]:
def jaccard_distance(a, b):
    if not isinstance(a, set):
        a = set(a)
    if not isinstance(b, set):
        b = set(b)
    union_size = len(a | b)
    inter_size = len(a & b)
    return (union_size - inter_size) / union_size

In [613]:
from datetime import datetime

result_files = glob(join(RESULTS_PATH, '*_*.json'))
result_files = [x for x in result_files if os.path.basename(x).startswith(('anova', 'infogain', 'rfe', 'chi2', 'mrmr'))]

from collections import defaultdict

# clf = 'svm'
# filter_ = 'anova'
# data = MDD
# target = 'stress'

# constraints = [('clf', KER_SVM), ('filter', 'infogain_10'), ('data', MDD), ('target', 'stress')]
constraints = [('clf', 'en'), ('filter', 'chi2'), ('data', EPI), ('tissue', WB)]
keyby = ['n_features']
i = 0
no_clf = 0
no_cond = 0
results = defaultdict(list)

d0 = datetime.now()
for f in result_files:
    i += 1
    exp_id = f.split('_')[-1].rstrip('.json')
    try:
        d = json.load(open(join(RESULTS_PATH, f), 'r'))
    except ValueError as e:
        if os.path.getsize(f) == 0:
            e = 'File size is 0. Removing.'
            os.remove(f)
        print('{} -> {}'.format(f, e))
        os.remove(f)
    base = {'exp_id': exp_id}
    left = []
    for k, v in d.items():
        if k != 'experiments':
            base[k] = v
    if os.path.basename(f).startswith('mrmr'):
        base['filter'] = 'mrmr'
    elif os.path.basename(f).startswith('rfe'):
        base['filter'] = 'rfe'
  
    if 'clf' not in base:
        no_clf += 1
        print(i, no_clf, no_cond)
        continue
        
    if not all(base[k] == v for k, v in constraints):
        no_cond += 1
        continue
        
    for exp in d['experiments']:
        it = exp['iteration']
        if 'subsets' not in exp:
            print('{} -> Field "subsets" not found.'.format(f))
            continue
        for s in exp['subsets']:
            if 'n_features' not in s:
                s['n_features'] = len(s['features'])
            key = tuple(base[x] if x in base else s[x] for x in keyby)
            results[key].append(set(s['features']))

print(datetime.now() - d0)
print(i, no_clf, no_cond)

32 1 31
36 2 34
370 3 357
423 4 409
458 5 443
0:12:42.236835
760 5 745


In [614]:
%%time
from itertools import combinations

MDD_SIZES = [485577, 200000, 100000, 50000, 10000, 5000, 1000, 500, 100, 10]

robust = defaultdict(dict)
for k in results:
    n_feat, = k
    if n_feat not in MDD_SIZES:
        continue
    ss = results[k]
    union = set()
    inter = ss[0]
    for s in ss:
        union |= s
        inter &= s
    robust[n_feat]['union'] = union
    robust[n_feat]['inter'] = inter
    jac = 0.0
    n = 0
    for i, j in combinations(xrange(len(ss)), 2):
        jac += jaccard_distance(ss[i], ss[j])
        n += 1
    jac /= n
    robust[n_feat]['jaccard'] = jac

CPU times: user 36.8 s, sys: 1.53 s, total: 38.4 s
Wall time: 39.3 s


In [615]:
SOMEWHERE = '../../selection/'
from os.path import join
import cPickle as pickle

# FILES = ['mdd_svm_anova.json', 'mdd_svm_mrmr.json', 'mdd_svm_infogain_10.json']
FILE1 = 'ad_ec_en_chi2.json'
FILE2 = 'ad_fc_svm_infogain_10.json'
FILE3 = 'ad_wb_en_chi2.json'

In [616]:
pickle.dump(robust, open(join(SOMEWHERE, FILE3), 'w'))

In [617]:
foo = pickle.load(open(join(SOMEWHERE, FILE3), 'r'))

In [618]:
foo.keys()

[100000, 200000, 100, 5000, 485577, 10, 50000, 1000, 500, 10000]

In [543]:
def robust_to_list_of_dicts(robust, clf, filter_):
     return[{'clf': clf, 'filter': filter_, 'union': len(v['union']), 'inter': len(v['inter']), 'jaccard': v['jaccard'], 'size': k} for k, v in robust.items()]

In [622]:
foo = []
foo += robust_to_list_of_dicts(pickle.load(open(join(SOMEWHERE, FILE1), 'r')), 'ec en', 'chi2')
foo += robust_to_list_of_dicts(pickle.load(open(join(SOMEWHERE, FILE2), 'r')), 'fc svm', 'infogain_10')
foo += robust_to_list_of_dicts(pickle.load(open(join(SOMEWHERE, FILE3), 'r')), 'wb en', 'chi2')
foo

[{'clf': 'ec en',
  'filter': 'chi2',
  'inter': 26783,
  'jaccard': 0.5705458560390116,
  'size': 100000,
  'union': 223182},
 {'clf': 'ec en',
  'filter': 'chi2',
  'inter': 74623,
  'jaccard': 0.4663780811346781,
  'size': 200000,
  'union': 359554},
 {'clf': 'ec en',
  'filter': 'chi2',
  'inter': 5,
  'jaccard': 0.8285991033448942,
  'size': 100,
  'union': 448},
 {'clf': 'ec en',
  'filter': 'chi2',
  'inter': 571,
  'jaccard': 0.7530691850919499,
  'size': 5000,
  'union': 18090},
 {'clf': 'ec en',
  'filter': 'chi2',
  'inter': 485577,
  'jaccard': 0.0,
  'size': 485577,
  'union': 485577},
 {'clf': 'ec en',
  'filter': 'chi2',
  'inter': 0,
  'jaccard': 0.9165099568092339,
  'size': 10,
  'union': 57},
 {'clf': 'ec en',
  'filter': 'chi2',
  'inter': 10297,
  'jaccard': 0.6372671043657417,
  'size': 50000,
  'union': 130681},
 {'clf': 'ec en',
  'filter': 'chi2',
  'inter': 83,
  'jaccard': 0.79544437721945,
  'size': 1000,
  'union': 4079},
 {'clf': 'ec en',
  'filter': 'chi2

In [637]:
def format_jaccard(jaccard):
    return '{:0.2f}'.format(jaccard)
format_jaccard = np.vectorize(format_jaccard)

df = pd.DataFrame(foo)
df['jaccard'] = format_jaccard(df['jaccard'])
## df = df.groupby(['filter', 'size'])
# df = df.drop('clf', axis=1)
## df = df.groupby(['filter'])
## df = df.unstack('filter')
df = df.set_index(['clf', 'filter', 'size']).unstack('clf').unstack('filter')
df = df.dropna(axis=1, how='all')
df = df.swaplevel(0, 2, axis=1)
df = df.swaplevel(0, 1, axis=1)
# df = df[[('anova', 'inter'), ('anova', 'union'), ('anova', 'jaccard'),
#          ('mrmr', 'inter'), ('mrmr', 'union'), ('mrmr', 'jaccard'),
#          ('infogain_10', 'inter'), ('infogain_10', 'union'), ('infogain_10', 'jaccard')]]
df = df[[('ec en', 'chi2', 'inter'), ('ec en', 'chi2', 'union'), ('ec en', 'chi2', 'jaccard'),
         ('fc svm', 'infogain_10', 'inter'), ('fc svm', 'infogain_10', 'union'), ('fc svm', 'infogain_10', 'jaccard'),
         ('wb en', 'chi2', 'inter'), ('wb en', 'chi2', 'union'), ('wb en', 'chi2', 'jaccard')]]
df = df.iloc[::-1]
foo_latex = df.to_latex(na_rep=' ')
foo_latex = process_latex(foo_latex, remove_lines=r'^size*')
repl = [
#     ('clf', ['b', '3r', 'c']),
#         ('en', 'bc'),
#         ('svm', 'bc'),
#         ('l1 svm', 'bc'),
        ('IG1', 'bc'),
        ('IG2', 'bc'),
        ('anova', 'bc'), ('mrmr', 'bc'),
        ('chi2', 'bc'), 
        ('pm', None)]
foo_latex = batch_replace(foo_latex, repl)
foo_latex = foo_latex.replace('filter', '')
foo_latex = my_replace(foo_latex, src='clf', dst='\# Feat', effects=['b', '2r', 'c'])
foo_latex = my_replace(foo_latex, src='inter', dst=r'$\bm{\cap}$', effects='c')
foo_latex = my_replace(foo_latex, src='union', dst=r'$\bm{\cup}$', effects='c')
foo_latex = my_replace(foo_latex, src='jaccard', dst=r'$\bm{J}$', effects='c')
foo_latex = add_clines(foo_latex, [(2, 2, 4), (2, 5, 7), (2, 8, 10)])
save_table(foo_latex, 'ad_robust.tex')
df

clf      ec en                      fc svm                   wb en          \
filter    chi2                 infogain_10                    chi2           
         inter   union jaccard       inter   union jaccard   inter   union   
size                                                                         
485577  485577  485577    0.00      485577  485577    0.00  485577  485577   
200000   74623  359554    0.47       71169  361948    0.47   72738  368988   
100000   26783  223182    0.57       23202  237819    0.60   25919  232786   
50000    10297  130681    0.64        8582  143783    0.67    9878  137432   
10000     1325   33487    0.73        1134   37417    0.76    1201   35704   
5000       571   18090    0.75         502   20104    0.78     476   19542   
1000        83    4079    0.80          72    4574    0.82      61    4592   
500         38    2145    0.81          28    2441    0.84      23    2420   
100          5     448    0.83           6     525    0.86       3     535   
10           0      57    0.92           0      51    0.87       0      68   

clf             
filter          
       jaccard  
size            
485577    0.00  
200000    0.48  
100000    0.59  
50000     0.66  
10000     0.75  
5000      0.78  
1000      0.83  
500       0.85  
100       0.88  
10        0.96

In [641]:
# anova = pickle.load(open(join(SOMEWHERE, 'mdd_svm_anova.json'), 'r'))
# mrmr = pickle.load(open(join(SOMEWHERE, 'mdd_svm_mrmr.json'), 'r'))
# ig = pickle.load(open(join(SOMEWHERE, 'mdd_svm_infogain_10.json'), 'r'))

chi2 = pickle.load(open(join(SOMEWHERE, FILE1), 'r'))
ig = pickle.load(open(join(SOMEWHERE, FILE2), 'r'))

In [645]:
chi2.keys()

[100000, 200000, 100, 5000, 485577, 10, 50000, 1000, 50, 500, 10000]

In [647]:
sorted(ig.keys())

[10, 100, 500, 1000, 5000, 10000, 50000, 100000, 200000, 485577]

In [667]:
robust_comb = []
probes = []
for size in [10, 100, 500, 1000, 5000, 10000, 50000]:#, 1000]:
#     union = anova[size]['union'] | mrmr[size]['union'] | ig[size]['union']
#     inter = anova[size]['inter'] & mrmr[size]['inter'] & ig[size]['inter']
    union = chi2[size]['union'] | ig[size]['union']
    inter = chi2[size]['inter'] & ig[size]['inter']
    probes.append(inter)
#     robust_comb[size]['union'] = len(union)
#     robust_comb[size]['inter'] = len(inter)
    robust_comb.append({'size': size, 'union': len(union), 'inter': len(inter)})

In [674]:
print(list(probes[5]))
for p in probes[5]:
    print(p)

[u'cg08639402', u'cg18865796', u'cg07930620', u'cg04193083', u'cg21604516', u'cg11450715', u'cg04529658', u'cg10233639', u'cg02214188', u'cg02466933', u'cg24088775', u'cg09789536', u'cg18641876', u'cg14993530', u'cg00306426', u'cg17847344', u'cg07450219', u'cg13042636']
cg08639402
cg18865796
cg07930620
cg04193083
cg21604516
cg11450715
cg04529658
cg10233639
cg02214188
cg02466933
cg24088775
cg09789536
cg18641876
cg14993530
cg00306426
cg17847344
cg07450219
cg13042636


In [668]:
probes_flatten = []
for p in probes:
    probes_flatten.extend(list(p))
print(len(probes_flatten))
print(len(set(probes_flatten)))
with open(join(SOMEWHERE, 'probes.txt'), 'w') as f:
    for p in probes_flatten:
        f.write(p + '\n')

367
345


In [669]:
foo = pd.DataFrame(robust_comb)
foo = foo.set_index('size')
foo = foo.iloc[::-1]
foo_latex = foo.to_latex(na_rep=' ')
foo_latex = process_latex(foo_latex, remove_lines=r'^size*')
foo_latex = my_replace(foo_latex, src='inter', dst=r'$\bm{\cap}$', effects='c')
foo_latex = my_replace(foo_latex, src='union', dst=r'$\bm{\cup}$', effects='c')
save_table(foo_latex, 'ad_robust_comb.tex')
foo

,inter,union
size,,
50000,345,234286
10000,18,67818
5000,4,37133
1000,0,8550
500,0,4549
100,0,970
10,0,108


## AD Robustness

In [ ]:
# EC+EN+chi2, FC+SVM+IG10

In [601]:
from datetime import datetime

result_files = glob(join(RESULTS_PATH, '*_*.json'))
result_files = [x for x in result_files if os.path.basename(x).startswith(('anova', 'infogain', 'rfe', 'chi2', 'mrmr'))]

from collections import defaultdict

# clf = 'svm'
# filter_ = 'anova'
# data = MDD
# target = 'stress'

constraints = [('clf', 'en'), ('filter', 'chi2'), ('data', EPI), ('tissue', EC)]
keyby = ['n_features']
i = 0
no_clf = 0
no_cond = 0
results = defaultdict(list)

d0 = datetime.now()
for f in result_files:
    i += 1
    exp_id = f.split('_')[-1].rstrip('.json')
    try:
        d = json.load(open(join(RESULTS_PATH, f), 'r'))
    except ValueError as e:
        if os.path.getsize(f) == 0:
            e = 'File size is 0. Removing.'
            os.remove(f)
        print('{} -> {}'.format(f, e))
        os.remove(f)
    base = {'exp_id': exp_id}
    left = []
    for k, v in d.items():
        if k != 'experiments':
            base[k] = v
    if os.path.basename(f).startswith('mrmr'):
        base['filter'] = 'mrmr'
    elif os.path.basename(f).startswith('rfe'):
        base['filter'] = 'rfe'
  
    if 'clf' not in base:
        no_clf += 1
        print(i, no_clf, no_cond)
        continue
        
    if not all(base[k] == v for k, v in constraints):
        no_cond += 1
        continue
        
    for exp in d['experiments']:
        it = exp['iteration']
        if 'subsets' not in exp:
            print('{} -> Field "subsets" not found.'.format(f))
            continue
        for s in exp['subsets']:
            if 'n_features' not in s:
                s['n_features'] = len(s['features'])
            key = tuple(base[x] if x in base else s[x] for x in keyby)
            results[key].append(set(s['features']))

print(datetime.now() - d0)
print(i, no_clf, no_cond)

32 1 31
36 2 34
370 3 357
423 4 409
458 5 443
0:12:44.303732
760 5 745
